In [1]:
#import required libraries
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_DETERMINISTIC_OPS'] = '1'
import cv2
import numpy as np
from imutils import paths
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report, roc_auc_score, matthews_corrcoef, precision_score, f1_score, recall_score
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, SimpleRNN,Dropout
from tensorflow.keras.utils import to_categorical
import pandas as pd
import time
import psutil
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
import keras

tf.keras.utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()

2024-04-04 06:10:15.852287: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-04 06:10:15.852353: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-04 06:10:15.855985: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-04 06:10:15.877827: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-04 06:10:18.553698: W ten

In [2]:
#!pip install tensorflow-tensorrt


In [3]:
#!pip install opencv-python
#!pip install imutils
#!pip install scikit-learn
#!pip install xgboost


In [4]:
#Class to load the dataset images from device

class SimpleDatasetLoader:
    # Method: Constructor
    def __init__(self, preprocessors=None):
        """
        :param preprocessors: List of image preprocessors
        """
        self.preprocessors = preprocessors

        if self.preprocessors is None:
            self.preprocessors = []

    # Method: Used to load a list of images for pre-processing
    def load(self, image_paths, verbose=-1):
        """
        :param image_paths: List of image paths
        :param verbose: Parameter for printing information to console
        :return: Tuple of data and labels
        """
        data, labels = [], []

        for i, image_path in enumerate(image_paths):
            image = cv2.imread(image_path)
            label = image_path.split(os.path.sep)[-2]

            if self.preprocessors is not None:
                for p in self.preprocessors:
                    image = p.preprocess(image)

            data.append(image)
            labels.append(label)

            if verbose > 0 and i > 0 and (i+1) % verbose == 0:
                print('[INFO]: Processed {}/{}'.format(i+1, len(image_paths)))

        return (np.array(data), np.array(labels))

In [5]:
#Class Preprocessror 
class SimplePreprocessor:
    # Method: Constructor
    def __init__(self, width, height, interpolation=cv2.INTER_AREA):
        """
        :param width: Image width
        :param height: Image height
        :param interpolation: Interpolation algorithm
        """
        self.width = width
        self.height = height
        self.interpolation = interpolation

    # Method: Used to resize the image to a fixed size (ignoring the aspect ratio)
    def preprocess(self, image):
        """
        :param image: Image
        :return: Re-sized image
        """
        return cv2.resize(image, (self.width, self.height), interpolation=self.interpolation)

In [6]:
from imutils import paths
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from __main__ import SimplePreprocessor
from __main__ import SimpleDatasetLoader

In [7]:
# Function to load and preprocess data using SimpleDatasetLoader
def load_and_preprocess_data(image_paths, target_size):
    sp = SimplePreprocessor(target_size[0], target_size[1])
    sdl = SimpleDatasetLoader(preprocessors=[sp])
    data, labels = sdl.load(image_paths, verbose=1000)

    print(data.shape)
    print(labels.shape)

    # Convert labels to one-hot encoding
    le = LabelEncoder()
    labels = le.fit_transform(labels)
    #labels = to_categorical(labels)

    return data, labels



# Get list of image paths
image_paths = list(paths.list_images("../melanoma_cancer_dataset"))

# Define target size for images
target_size = (224, 224)  # Change this to your desired size

# Load and preprocess data
print('[INFO]: Images loading....')
data, labels = load_and_preprocess_data(image_paths, target_size)

[INFO]: Images loading....
[INFO]: Processed 1000/9605
[INFO]: Processed 2000/9605
[INFO]: Processed 3000/9605
[INFO]: Processed 4000/9605
[INFO]: Processed 5000/9605
[INFO]: Processed 6000/9605
[INFO]: Processed 7000/9605
[INFO]: Processed 8000/9605
[INFO]: Processed 9000/9605
(9605, 224, 224, 3)
(9605,)


In [8]:
from tensorflow.keras.applications import  ResNet50, EfficientNetB0
from tensorflow.keras.models import Model
with tf.device('/CPU:0'):
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(target_size[0], target_size[1], 3))
    model = Model(inputs=base_model.input, outputs=base_model.output)
    features = model.predict(data, batch_size=32, verbose=1)

# Flatten the features
features_flatten = features.reshape(features.shape[0], -1)
    
print(features_flatten.shape)

2024-04-04 06:11:31.083178: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38380 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:65:00.0, compute capability: 8.0


301/301 [==============================] - 344s 1s/step
(9605, 62720)


In [9]:
data = features_flatten

In [10]:
import gc
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
#tf.config.experimental.set_memory_growth(tf.config.list_physical_devices('GPU')[0], True)
#tf.config.experimental.enable_tensor_float_32_execution(True)
#tf.config.threading.set_inter_op_parallelism_threads(1)
#tf.config.threading.set_intra_op_parallelism_threads(1)
# Initialize a DataFrame to store results
results_df = pd.DataFrame(columns=['Model', 'Average Accuracy', 'Average Sensitivity', 'Average Specificity',
                                   'Average AUC-ROC', 'Average MCC', 'Average Precision', 'Average F1 Score',
                                   'Memory Used (MB)', 'Time (s)'])

# Function to evaluate a model
def evaluate_model(model, name, data, labels):
    if name == "VGG16":
        labels = to_categorical(labels)

    elif name == "VGG19":  
        labels = to_categorical(labels)
    else:
        data = data
    accuracy_list = []
    sensitivity_list = []
    specificity_list = []
    auc_roc_list = []
    mcc_list = []
    precision_list = []
    f1_list = []
    time_start = time.time()
    memory_start = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2

    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
    if name == "VGG16" or name == "VGG19":


        with tf.device('/GPU:0'):
            tf.random.set_seed(0)
            for fold, (train_index, test_index) in enumerate(skf.split(data, labels.argmax(axis=1))):
                print(f'\n[INFO] Fold {fold + 1} / 10 for {name}')

                X_train, X_test = data[train_index], data[test_index]
                y_train, y_test = labels[train_index], labels[test_index]


                # Adding early stopping to prevent overfitting
                early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)
                keras.utils.set_random_seed(0)
                np.random.seed(0)
                tf.random.set_seed(0)
                #tf.config.run_functions_eagerly(True)
                tf.data.experimental.enable_debug_mode()
                # Train the model
                # Example of using tf.data.Dataset
                train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
                train_dataset = train_dataset.shuffle(buffer_size=len(X_train)).batch(64)

                model.fit(train_dataset, epochs=20, callbacks=[early_stopping])

                # Evaluate the model
                predictions = model.predict(X_test)
                y_pred = np.argmax(predictions, axis=1)
                y_test_encoded = np.argmax(y_test, axis=1)

                
                # Calculate evaluation metrics for the current fold
                accuracy = np.mean(y_pred == y_test_encoded)
                sensitivity = recall_score(y_test_encoded, y_pred, pos_label=1)
                specificity = recall_score(y_test_encoded, y_pred, pos_label=0)
                auc_roc = roc_auc_score(y_test_encoded, y_pred)
                mcc = matthews_corrcoef(y_test_encoded, y_pred)
                precision = precision_score(y_test_encoded, y_pred, pos_label=1)
                f1 = f1_score(y_test_encoded, y_pred, pos_label=1)
                # Append metrics to lists
                accuracy_list.append(accuracy)
                sensitivity_list.append(sensitivity)
                specificity_list.append(specificity)
                auc_roc_list.append(auc_roc)
                mcc_list.append(mcc)
                precision_list.append(precision)
                f1_list.append(f1)

                # Print metrics for the current fold
                print(f"Accuracy: {accuracy}")
                print(f"Sensitivity: {sensitivity}")
                print(f"Specificity: {specificity}")
                print(f"AUC-ROC: {auc_roc}")
                print(f"MCC: {mcc}")
                print(f"Precision: {precision}")
                print(f"F1 Score: {f1}")
                gc.collect()

            # Calculate average metrics
            average_accuracy = np.mean(accuracy_list)
            average_sensitivity = np.mean(sensitivity_list)
            average_specificity = np.mean(specificity_list)
            average_auc_roc = np.mean(auc_roc_list)
            average_mcc = np.mean(mcc_list)
            average_precision = np.mean(precision_list)
            average_f1 = np.mean(f1_list)

            time_end = time.time()
            memory_end = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2
            del X_train, X_test, y_train, y_test
            gc.collect()
            tf.keras.backend.clear_session()
    else:
        for fold, (train_index, test_index) in enumerate(skf.split(data, labels)):
            print(f'\n[INFO] Fold {fold + 1} / 10 for {name}')

            X_train, X_test = data[train_index], data[test_index]
            y_train, y_test = labels[train_index], labels[test_index]

            # Train the model
            model.fit(X_train, y_train)

            # Evaluate the model
            predictions = model.predict(X_test)
            y_pred = predictions
            y_test_encoded = y_test

            # Calculate evaluation metrics for the current fold
            accuracy = np.mean(y_pred == y_test_encoded)
            sensitivity = recall_score(y_test, y_pred, pos_label=1)
            specificity = recall_score(y_test, y_pred, pos_label=0)
            auc_roc = roc_auc_score(y_test, predictions)
            mcc = matthews_corrcoef(y_test, y_pred)
            precision = precision_score(y_test, y_pred, pos_label=1)
            f1 = f1_score(y_test, y_pred, pos_label=1)
            # Append metrics to lists
            accuracy_list.append(accuracy)
            sensitivity_list.append(sensitivity)
            specificity_list.append(specificity)
            auc_roc_list.append(auc_roc)
            mcc_list.append(mcc)
            precision_list.append(precision)
            f1_list.append(f1)

            # Print metrics for the current fold
            print(f"Accuracy: {accuracy}")
            print(f"Sensitivity: {sensitivity}")
            print(f"Specificity: {specificity}")
            print(f"AUC-ROC: {auc_roc}")
            print(f"MCC: {mcc}")
            print(f"Precision: {precision}")
            print(f"F1 Score: {f1}")
            gc.collect()

        # Calculate average metrics
        average_accuracy = np.mean(accuracy_list)
        average_sensitivity = np.mean(sensitivity_list)
        average_specificity = np.mean(specificity_list)
        average_auc_roc = np.mean(auc_roc_list)
        average_mcc = np.mean(mcc_list)
        average_precision = np.mean(precision_list)
        average_f1 = np.mean(f1_list)

        time_end = time.time()
        memory_end = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2
        

    # Append results to DataFrame
    results_df.loc[len(results_df)] = [name, average_accuracy, average_sensitivity, average_specificity,
                                       average_auc_roc, average_mcc, average_precision, average_f1,
                                       memory_end - memory_start, time_end - time_start]





In [11]:
from sklearn.linear_model import LogisticRegression
# SVM
lr_model = LogisticRegression(n_jobs=-1, random_state=0)
evaluate_model(lr_model, 'LR', data, labels)




[INFO] Fold 1 / 10 for LR


/opt/tljh/user/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.931321540062435
Sensitivity: 0.928416485900217
Specificity: 0.934
AUC-ROC: 0.9312082429501084
MCC: 0.8624164859002169
Precision: 0.928416485900217
F1 Score: 0.928416485900217

[INFO] Fold 2 / 10 for LR


/opt/tljh/user/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.9261186264308012
Sensitivity: 0.9240780911062907
Specificity: 0.928
AUC-ROC: 0.9260390455531454
MCC: 0.8520078634174864
Precision: 0.922077922077922
F1 Score: 0.923076923076923

[INFO] Fold 3 / 10 for LR
Accuracy: 0.9334027055150884
Sensitivity: 0.9154013015184381
Specificity: 0.95
AUC-ROC: 0.932700650759219
MCC: 0.8667975753493689
Precision: 0.9440715883668904
F1 Score: 0.9295154185022027

[INFO] Fold 4 / 10 for LR
Accuracy: 0.9073881373569199
Sensitivity: 0.9002169197396963
Specificity: 0.914
AUC-ROC: 0.9071084598698482
MCC: 0.8144395518682794
Precision: 0.9061135371179039
F1 Score: 0.9031556039173014

[INFO] Fold 5 / 10 for LR
Accuracy: 0.9157127991675338
Sensitivity: 0.8893709327548807
Specificity: 0.94
AUC-ROC: 0.9146854663774403
MCC: 0.8316470960715556
Precision: 0.9318181818181818
F1 Score: 0.9100998890122086

[INFO] Fold 6 / 10 for LR
Accuracy: 0.9177083333333333
Sensitivity: 0.9
Specificity: 0.934
AUC-ROC: 0.9169999999999998
MCC: 0.8352520005784769
Precision: 0.926

In [12]:
#NB
nb_model = GaussianNB()
evaluate_model(nb_model, 'NB', data, labels)


[INFO] Fold 1 / 10 for NB
Accuracy: 0.8064516129032258
Sensitivity: 0.6919739696312365
Specificity: 0.912
AUC-ROC: 0.8019869848156183
MCC: 0.6223716217629179
Precision: 0.8787878787878788
F1 Score: 0.7742718446601942

[INFO] Fold 2 / 10 for NB
Accuracy: 0.813735691987513
Sensitivity: 0.7440347071583514
Specificity: 0.878
AUC-ROC: 0.8110173535791757
MCC: 0.629552281361068
Precision: 0.849009900990099
F1 Score: 0.7930635838150288

[INFO] Fold 3 / 10 for NB
Accuracy: 0.8251821019771072
Sensitivity: 0.7483731019522777
Specificity: 0.896
AUC-ROC: 0.8221865509761388
MCC: 0.6537896828099204
Precision: 0.8690176322418136
F1 Score: 0.8041958041958043

[INFO] Fold 4 / 10 for NB
Accuracy: 0.8293444328824141
Sensitivity: 0.7613882863340564
Specificity: 0.892
AUC-ROC: 0.8266941431670282
MCC: 0.6610615564646122
Precision: 0.8666666666666667
F1 Score: 0.8106235565819861

[INFO] Fold 5 / 10 for NB
Accuracy: 0.8012486992715921
Sensitivity: 0.6963123644251626
Specificity: 0.898
AUC-ROC: 0.7971561822125

In [13]:

# KNN
knn_model = KNeighborsClassifier()
evaluate_model(knn_model, 'KNN', data, labels)




[INFO] Fold 1 / 10 for KNN
Accuracy: 0.8959417273673257
Sensitivity: 0.8221258134490239
Specificity: 0.964
AUC-ROC: 0.8930629067245119
MCC: 0.79761390515272
Precision: 0.9546599496221663
F1 Score: 0.8834498834498836

[INFO] Fold 2 / 10 for KNN
Accuracy: 0.9011446409989594
Sensitivity: 0.8373101952277657
Specificity: 0.96
AUC-ROC: 0.8986550976138828
MCC: 0.806405115282701
Precision: 0.9507389162561576
F1 Score: 0.8904267589388696

[INFO] Fold 3 / 10 for KNN
Accuracy: 0.9042663891779397
Sensitivity: 0.8286334056399133
Specificity: 0.974
AUC-ROC: 0.9013167028199566
MCC: 0.8149780887442145
Precision: 0.9670886075949368
F1 Score: 0.8925233644859814

[INFO] Fold 4 / 10 for KNN
Accuracy: 0.8928199791883454
Sensitivity: 0.8221258134490239
Specificity: 0.958
AUC-ROC: 0.8900629067245119
MCC: 0.7906580317755493
Precision: 0.9475
F1 Score: 0.8803716608594658

[INFO] Fold 5 / 10 for KNN
Accuracy: 0.8668054110301769
Sensitivity: 0.7787418655097614
Specificity: 0.948
AUC-ROC: 0.8633709327548805
MCC:

In [14]:
# Random Forest
rf_model = RandomForestClassifier(random_state=0, n_jobs=-1)
evaluate_model(rf_model, 'Random Forest', data, labels)



[INFO] Fold 1 / 10 for Random Forest
Accuracy: 0.9073881373569199
Sensitivity: 0.8893709327548807
Specificity: 0.924
AUC-ROC: 0.9066854663774403
MCC: 0.8145662714112825
Precision: 0.9151785714285714
F1 Score: 0.902090209020902

[INFO] Fold 2 / 10 for Random Forest
Accuracy: 0.9001040582726326
Sensitivity: 0.8915401301518439
Specificity: 0.908
AUC-ROC: 0.8997700650759219
MCC: 0.7998386071892671
Precision: 0.899343544857768
F1 Score: 0.8954248366013073

[INFO] Fold 3 / 10 for Random Forest
Accuracy: 0.9094693028095734
Sensitivity: 0.8872017353579176
Specificity: 0.93
AUC-ROC: 0.9086008676789588
MCC: 0.818894564524596
Precision: 0.9211711711711712
F1 Score: 0.9038674033149171

[INFO] Fold 4 / 10 for Random Forest
Accuracy: 0.9094693028095734
Sensitivity: 0.8893709327548807
Specificity: 0.928
AUC-ROC: 0.9086854663774403
MCC: 0.8188108904933332
Precision: 0.9192825112107623
F1 Score: 0.9040793825799338

[INFO] Fold 5 / 10 for Random Forest
Accuracy: 0.899063475546306
Sensitivity: 0.8763557

In [15]:
import lightgbm
# LGBM
lgb_model = lightgbm.LGBMClassifier(random_state=0, n_jobs = -1)
evaluate_model(lgb_model, 'LGBM', data, labels)


[INFO] Fold 1 / 10 for LGBM
[LightGBM] [Info] Number of positive: 4144, number of negative: 4500
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 16.582867 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 15993600
[LightGBM] [Info] Number of data points in the train set: 8644, number of used features: 62720
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.479408 -> initscore=-0.082416
[LightGBM] [Info] Start training from score -0.082416
Accuracy: 0.922996878251821
Sensitivity: 0.9045553145336226
Specificity: 0.94
AUC-ROC: 0.9222776572668111
MCC: 0.8459179545971256
Precision: 0.9328859060402684
F1 Score: 0.9185022026431718

[INFO] Fold 2 / 10 for LGBM
[LightGBM] [Info] Number of positive: 4144, number of negative: 4500
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 17.083087 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Tota

In [16]:
# Save results to a CSV file
results_df.to_csv('DS4_VGG16.csv', index=False)

In [17]:
results_df

,Model,Average Accuracy,Average Sensitivity,Average Specificity,Average AUC-ROC,Average MCC,Average Precision,Average F1 Score,Memory Used (MB),Time (s)
0,LR,0.922957,0.913792,0.9314,0.922596,0.845739,0.924718,0.919157,939.632812,2113.084215
1,NB,0.817388,0.726382,0.9012,0.813791,0.640348,0.871537,0.792039,2298.394531,130.890037
2,KNN,0.889014,0.813676,0.9584,0.886038,0.783787,0.947415,0.875394,2309.218750,225.799369
3,Random Forest,0.906716,0.886862,0.9250,0.905931,0.813361,0.916039,0.901153,2337.269531,1037.558526
4,LGBM,0.920667,0.902067,0.9378,0.919934,0.841311,0.930395,0.915960,14090.023438,9758.549103
